We hebben alle `nvt` en `?` kolomen verwijderd. Dit hebben we gedaan omdat er tegen ons gezegd is dat deze niet interresand zijn. Daarnaast hebben we ook de regels met prioriteit `9` verwijderd. Dit komt omdat deze regels adimistratief zijn en geen functie herstel bevatten.

Vervolgens hebben we nog een aantal kolomen ook nog verwijderd, die worden hieronder verder toegeligt:

`stm_pplg_van` en `stm_pplg_naar`. Wij hebben hier niks aan. Er worden namelijk zowel getallen als strings door elkaar gebruikt. Dit is niet handig voor zo goed als elke model. Daarom hebben we deze kolomen verwijderd.

Ook hebben wij de kolommen `stm_equipm_omschr_gst`, `stm_equipm_omschr_mld`, `stm_sap_meldtekst`, `stm_sap_meldtekst_lang`, `stm_oorz_tkst`, `stm_oorz_tekst_kort`, `stm_schade_verhaalb_jn` verwijderd. Dit komt omdat deze beschrijvingen zijn en dus niet bruikbaar zijn voor een model.

Daarnaast hebben we ook nog de volgende kolommen verwijderd: `stm_equipm_nr_gst`,  `stm_equipm_soort_gst`, `#stm_sap_meldnr`, `stm_equipm_nr_mld`, `stm_equipm_soort_mld`. Dit komt omdat deze kolommen niet relevant zijn voor ons model omdat ze geen informatie toevoegen.

`stm_vl_post` is verwijderd omdat hier veel values missen en de *informatie al hebben in andere kolommen* (niet excact dezelfde informatie maar `stm_vl_post` komt wel in de buurt).

Wij hebben als feuture variabelen de volgende gekozen: `stm_geo_mld`, `stm_prioriteit`, `stm_oorz_code`, `stm_contractgeb_gst`

Over de volgende feuture variabelen twijfelen we nog of ze nuttig zijn: `stm_sap_meld_ddt`, `stm_km_van_mld`, `stm_km_tot_mld` en `stm_techn_mld` Hierom gaan we aan deze extra aandacht besteden door ze 1 voor 1 eruit te halen en te kijken of ze het model beinvloeden.

Onze target variabele is het verschil tussen de tijd dat de aannemer een prognose geeft (``)

In [2]:
import pandas as pd
from datetime import timedelta
from sklearn.preprocessing import LabelEncoder
def clean_data(df):
    # Drop duplicate rows in columns: '#stm_sap_meldnr', 'stm_mon_nr' and 58 other columns
    df = df.drop_duplicates(subset=['#stm_sap_meldnr', 'stm_mon_nr', 'stm_vl_post', 'stm_sap_meld_ddt', 'stm_sap_meldtekst_lang', 'stm_sap_meldtekst', 'stm_geo_mld', 'stm_equipm_nr_mld', 'stm_equipm_soort_mld', 'stm_equipm_omschr_mld', 'stm_km_van_mld', 'stm_km_tot_mld', 'stm_prioriteit', 'stm_aanngeb_ddt', 'stm_oh_pg_gst', 'stm_geo_gst', 'stm_equipm_nr_gst', 'stm_equipm_soort_gst', 'stm_equipm_omschr_gst', 'stm_km_van_gst', 'stm_km_tot_gst', 'stm_oorz_groep', 'stm_oorz_code', 'stm_oorz_tkst', 'stm_fh_ddt', 'stm_fh_status', 'stm_sap_storeind_ddt', 'stm_tao_indicator', 'stm_tao_indicator_vorige', 'stm_tao_soort_mutatie', 'stm_tao_telling_mutatie', 'stm_tao_beinvloedbaar_indicator', 'stm_sap_melddatum', 'stm_sap_meldtijd', 'stm_contractgeb_mld', 'stm_techn_mld', 'stm_contractgeb_gst', 'stm_techn_gst', 'stm_aanngeb_dd', 'stm_aanngeb_tijd', 'stm_aanntpl_dd', 'stm_aanntpl_tijd', 'stm_progfh_in_datum', 'stm_progfh_in_tijd', 'stm_progfh_in_invoer_dat', 'stm_progfh_in_invoer_tijd', 'stm_progfh_in_duur', 'stm_progfh_gw_tijd', 'stm_progfh_gw_teller', 'stm_fh_dd', 'stm_fh_tijd', 'stm_fh_duur', 'stm_sap_storeinddatum', 'stm_sap_storeindtijd', 'stm_oorz_tekst_kort', 'stm_pplg_van', 'stm_pplg_naar', 'stm_dstrglp_van', 'stm_dstrglp_naar'])
    
    # Drop columns: 'stm_pplg_van', 'stm_pplg_naar' and 7 other columns
    df = df.drop(columns=['stm_pplg_van', 'stm_pplg_naar', 'stm_equipm_omschr_mld', 'stm_sap_meldtekst_lang', 'stm_sap_meldtekst', 'stm_equipm_omschr_gst', 'stm_oorz_tkst', 'stm_oorz_tekst_kort', 'stm_equipm_nr_gst', 'stm_equipm_soort_gst', '#stm_sap_meldnr', 'stm_vl_post', 'stm_equipm_nr_mld', 'stm_equipm_soort_mld'])
    # Drop first row
    df = df.iloc[1:, :]
    # Filter rows based on column: 'stm_prioriteit'
    df = df[df['stm_prioriteit'].isin([1.0, 2.0, 4.0, 5.0, 8.0])]
    # Drop rows with NaN values in column: 'stm_oorz_code'
    df = df.dropna(subset=['stm_oorz_code'])
    
    # Filter not valid integer values in column: 'stm_geo_mld'
    df['stm_geo_mld'] = pd.to_numeric(df['stm_geo_mld'], errors="coerce")
    df.dropna(subset=['stm_geo_mld'], inplace=True)

    # Create a new column with the duration between the time a prognose is made and the time the "functie herstel" is done
    df['stm_progfh_in_invoer_dat'] = df['stm_progfh_in_invoer_dat'].astype(str)
    df['duur_prog_fh'] = ((pd.to_datetime(df['stm_progfh_in_invoer_dat'] + ' ' + df['stm_progfh_in_invoer_tijd'], errors='coerce')) - (pd.to_datetime(df['stm_fh_ddt'], errors='coerce'))) # There is also a stm_progfh_in_invoer_ddt column but since it was marked as N.V.T its not inside the csv anymore :/
    df.dropna(subset=['duur_prog_fh'], inplace=True)

    # Filter rows that are shorter than 5 minutes and longer than 8 hours
    df = df[((timedelta(minutes=5) <= df['duur_prog_fh']) & (df['duur_prog_fh'] <= timedelta(hours=8)))]

    # Convert columns to numeric and drop the rows with NaN values
    df['stm_contractgeb_gst'] = pd.to_numeric(df['stm_contractgeb_gst'], errors="coerce")
    df.dropna(subset=['stm_contractgeb_gst'], inplace=True)
    
    # Convert columns to numeric with the label encoder and drop the rows with NaN values
    le = LabelEncoder()
    df['stm_techn_mld'] = le.fit_transform(df['stm_techn_mld'])
    df.dropna(subset=['stm_techn_mld'], inplace=True)
    df = df.astype({'stm_geo_mld':'float', 'stm_prioriteit':'float', 'stm_oorz_code':'float', 'stm_contractgeb_gst':'float', 'stm_techn_mld':'int', 'stm_sap_meld_ddt':'datetime64[ns]', 'stm_km_van_mld':'float', 'stm_km_tot_mld':'float'})
    # df = df.astype({'stm_geo_mld':'int', 'stm_prioriteit':'int', 'stm_oorz_code':'int', 'stm_contractgeb_gst':'int', 'stm_mon_nr':'int', 'stm_sap_meld_ddt':'datetime64[ns]'})
    return df

# df = pd.read_csv('dataset-prorail-clean-3.csv')
# df = clean_data(df)
# print(df['duur_prog_fh'].head())
# print(df.shape)

In [ ]:
def create_bins(df, column, num_bins):
    # Create bins for the column
    df[column + '_bins'], bin_edges = pd.qcut(df[column], q=num_bins, duplicates='drop', retbins=True)
    return df, bin_edges